# Ecoli Thames Data Processing

In [1]:
import os
import json
import pandas as pd
import bng_latlon as bl

### Specify the path(s) to the data

In [2]:
dir_new_samples_path = "/Users/massimilianoarca/Library/CloudStorage/OneDrive-PolitecnicodiMilano/Thames Ecoli/Reader Output"
dir_store_path = "/Users/massimilianoarca/Library/CloudStorage/OneDrive-PolitecnicodiMilano/Thames Ecoli/temporary results"
filename_new_samples = "merged_data.csv"
# file_old_samples_path = "/Users/massimilianoarca/Library/CloudStorage/OneDrive-PolitecnicodiMilano/SafeCREW/Thames Ecoli/historical_samples.csv"
file_old_samples_path = None

# paths to further features
manual_counting_path = "/Users/massimilianoarca/Library/CloudStorage/OneDrive-PolitecnicodiMilano/Thames Ecoli/further_features/Thames Sampling Ecoli - manual counting.xlsx"
water_quality_path = "/Users/massimilianoarca/Library/CloudStorage/OneDrive-PolitecnicodiMilano/Thames Ecoli/further_features/Thames Sampling Water Quality Data.xlsx"
historic_discharges_path = "/Users/massimilianoarca/Library/CloudStorage/OneDrive-PolitecnicodiMilano/Thames Ecoli/further_features/Thames Water Historic Discharges by site (missing fleet main before 2023).xlsx"

### Validate paths

In [3]:
if not os.path.isdir(dir_new_samples_path):
    raise ValueError(
        "The path to the folder containing the new samples excel files is not a directory"
    )

if not os.path.isdir(dir_store_path):
    raise ValueError(
        "The path to store the concatenated excel file is not a directory"
    )

if file_old_samples_path is not None:
    if not os.path.isfile(file_old_samples_path):
        raise ValueError(
            "The path to the file containing the old samples excel file is not a file"
        )

if not filename_new_samples.endswith(".csv"):
    raise ValueError("The name of the new file must include the csv extension")

## Load data

In [4]:
from genericpath import isdir


def append_data(new_data_folder, old_dataset_path):
    """
    Append the data in the new_data_folder to the old_dataset
    """

    new_samples = []
    for root, dirs, files in os.walk(new_data_folder):
        for file in files:
            if file.endswith(".xlsx"):
                new_samples.append(pd.read_excel(os.path.join(root, file)))
            elif file.endswith(".csv"):
                new_samples.append(pd.read_csv(os.path.join(root, file)))
            elif file.endswith(".DS_Store"):
                pass
            else:
                raise ValueError("The file extension is not supported")

        for dir in dirs:
            if isdir(os.path.join(root, dir)):
                new_samples.append(
                    append_data(os.path.join(root, dir), old_dataset_path)
                )

    new_samples_df = pd.concat(new_samples, ignore_index=True)
    if old_dataset_path is not None:
        if old_dataset_path.endswith(".xlsx"):
            old_samples_df = pd.read_excel(old_dataset_path)
        elif old_dataset_path.endswith(".csv"):
            old_samples_df = pd.read_csv(old_dataset_path)
        else:
            raise ValueError("The file extension is not supported")
        new_samples_df = pd.concat(
            [old_samples_df, new_samples_df], ignore_index=True
        )
    return new_samples_df

In [5]:
raw_reader_output_df = append_data(dir_new_samples_path, None)

In [6]:
raw_reader_output_df

,Technician,Sample ID,Image Date Time,Plate Type,Dilution,Barcode Text,Red With Gas Raw Count,Red With Gas Edited Count,Red With Gas Calculated Result,Red Without Gas Raw Count,Red Without Gas Edited Count,Red Without Gas Calculated Result,Blue With Gas Raw Count,Blue With Gas Edited Count,Blue With Gas Calculated Result,Blue Without Gas Raw Count,Blue Without Gas Edited Count,Blue Without Gas Calculated Result,Comments
0,Admin,20230905_1010_B1.1,10/13/2023 1:05:35 PM,ECCC,1:1,-,336,334,334,0,0,0,6,7,7,1,1,1,NaN
1,Admin,20230905_1010_B1.2,10/13/2023 1:06:12 PM,ECCC,1:1,-,365,364,364,0,0,0,8,9,9,0,0,0,NaN
2,Admin,20230905_1010_B2.1,10/13/2023 1:06:57 PM,ECCC,1:1,-,26,26,26,79,79,79,4,4,4,2,2,2,NaN
3,Admin,20230905_1010_B2.2,10/13/2023 1:07:50 PM,ECCC,1:1,-,43,43,43,107,107,107,10,10,10,3,2,2,NaN
4,Admin,20230905_1126_B1.1,10/13/2023 1:09:26 PM,ECCC,1:1,-,36,36,36,108,108,108,5,5,5,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3877,Admin,20230824_1405_D2.2,8/30/2023 2:41:47 PM,ECCC,1:1,-,60,52,52,120,128,128,15,15,15,2,2,2,NaN
3878,Admin,20230824_1420_E1.2,8/30/2023 2:43:18 PM,ECCC,1:1,-,30,25,25,91,96,96,6,6,6,3,3,3,NaN
3879,Admin,20230824_1420_E1.1,8/30/2023 2:45:21 PM,ECCC,1:1,-,45,34,34,84,95,95,9,9,9,1,1,1,NaN
3880,Admin,20230824_1420_E2.1,8/30/2023 2:46:30 PM,ECCC,1:1,-,58,49,49,93,102,102,4,4,4,1,1,1,NaN


In [7]:
raw_reader_output_df

,Technician,Sample ID,Image Date Time,Plate Type,Dilution,Barcode Text,Red With Gas Raw Count,Red With Gas Edited Count,Red With Gas Calculated Result,Red Without Gas Raw Count,Red Without Gas Edited Count,Red Without Gas Calculated Result,Blue With Gas Raw Count,Blue With Gas Edited Count,Blue With Gas Calculated Result,Blue Without Gas Raw Count,Blue Without Gas Edited Count,Blue Without Gas Calculated Result,Comments
0,Admin,20230905_1010_B1.1,10/13/2023 1:05:35 PM,ECCC,1:1,-,336,334,334,0,0,0,6,7,7,1,1,1,NaN
1,Admin,20230905_1010_B1.2,10/13/2023 1:06:12 PM,ECCC,1:1,-,365,364,364,0,0,0,8,9,9,0,0,0,NaN
2,Admin,20230905_1010_B2.1,10/13/2023 1:06:57 PM,ECCC,1:1,-,26,26,26,79,79,79,4,4,4,2,2,2,NaN
3,Admin,20230905_1010_B2.2,10/13/2023 1:07:50 PM,ECCC,1:1,-,43,43,43,107,107,107,10,10,10,3,2,2,NaN
4,Admin,20230905_1126_B1.1,10/13/2023 1:09:26 PM,ECCC,1:1,-,36,36,36,108,108,108,5,5,5,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3877,Admin,20230824_1405_D2.2,8/30/2023 2:41:47 PM,ECCC,1:1,-,60,52,52,120,128,128,15,15,15,2,2,2,NaN
3878,Admin,20230824_1420_E1.2,8/30/2023 2:43:18 PM,ECCC,1:1,-,30,25,25,91,96,96,6,6,6,3,3,3,NaN
3879,Admin,20230824_1420_E1.1,8/30/2023 2:45:21 PM,ECCC,1:1,-,45,34,34,84,95,95,9,9,9,1,1,1,NaN
3880,Admin,20230824_1420_E2.1,8/30/2023 2:46:30 PM,ECCC,1:1,-,58,49,49,93,102,102,4,4,4,1,1,1,NaN


In [8]:
raw_manual_counting_df = pd.read_excel(manual_counting_path)
raw_water_quality_df = pd.read_excel(water_quality_path)
raw_historic_discharges_df = pd.read_excel(
    historic_discharges_path, header=None
)

In [9]:
raw_manual_counting_df

,Counter,Date,Time,Site,Sample,Ecoli (1ml),Coliform (1ml)
0,Randa,2023-01-31,10:40:00,A,1.0,3.0,13.0
1,Randa,2023-01-31,10:40:00,A,2.0,3.0,7.0
2,Randa,2023-01-31,11:14:00,C,2.0,6.0,11.0
3,Randa,2023-01-31,11:15:00,C,1.0,8.0,11.0
4,Randa,2023-01-31,11:35:00,D,1.0,11.0,10.0
...,...,...,...,...,...,...,...
165,Alisha,2023-06-29,11:47:00,B,1.1,241.0,131.0
166,Alisha,2023-06-29,11:22:00,A,1.2,81.0,16.0
167,Alisha,2023-06-29,11:22:00,A,1.1,95.0,12.0
168,Alisha,2023-06-29,11:22:00,A,2.2,42.0,6.0


In [10]:
raw_water_quality_df

,FOLLOW UP,Sampler,Date,Time,Site,Sample,Temp C,Ph,Cond (ms)
0,NaN,Randa,2023-01-31,10:40:00,A,1,10.0,8.4,0.690000
1,NaN,Randa,2023-01-31,10:40:00,A,2,9.5,8.2,0.730000
2,NaN,Randa,2023-01-31,11:15:00,C,1,8.8,8.2,0.720000
3,NaN,Randa,2023-01-31,11:15:00,C,2,7.9,8.1,0.710000
4,X,Randa,2023-01-31,11:35:00,D,1,NaN,8.1,0.720000
...,...,...,...,...,...,...,...,...,...
691,NaN,NaN,2023-09-07,15:45:00,E,2,24.0,7.9,1.540000
692,X,NaN,2023-09-07,NaN,B,1,24.3,7.9,1.890000
693,X,NaN,2023-09-07,NaN,B,2,23.8,7.8,1.940000
694,X,NaN,2023-09-07,NaN,E,2,24.2,7.9,1.875000


In [11]:
raw_historic_discharges_df

,0
0,"{""meta"": {""publisher"": ""Thames Water Utilities..."
1,"{""meta"": {""publisher"": ""Thames Water Utilities..."
2,"{""meta"": {""publisher"": ""Thames Water Utilities..."
3,"{""meta"": {""publisher"": ""Thames Water Utilities..."
4,"{""meta"": {""publisher"": ""Thames Water Utilities..."
...,...
4171,"{""meta"": {""publisher"": ""Thames Water Utilities..."
4172,"{""meta"": {""publisher"": ""Thames Water Utilities..."
4173,"{""meta"": {""publisher"": ""Thames Water Utilities..."
4174,"{""meta"": {""publisher"": ""Thames Water Utilities..."


## Data Preprocessing

### Raw Automated Counting

In [12]:
reader_output_df = raw_reader_output_df.copy()

reader_output_df.drop_duplicates(inplace=True)

reader_output_df[["Date", "Time", "SiteSample"]] = raw_reader_output_df[
    "Sample ID"
].str.split("_", expand=True)

reader_output_df["Date"] = pd.to_datetime(
    reader_output_df["Date"], format="%Y%m%d"
).dt.date
reader_output_df["Time"].replace("XXXX", pd.NaT, inplace=True)  # type: ignore
reader_output_df["Time"] = pd.to_datetime(
    reader_output_df["Time"], format="%H%M", errors="coerce"
).dt.strftime("%H:%M")
reader_output_df["Image Date Time"] = pd.to_datetime(
    reader_output_df["Image Date Time"]
)

date_col = reader_output_df.pop("Date")
time_col = reader_output_df.pop("Time")

reader_output_df.insert(2, "Date", date_col)
reader_output_df.insert(3, "Time", time_col)

reader_output_df.insert(4, "Site", reader_output_df["SiteSample"].str.extract("([A-Za-z]+)", expand=True))  # type: ignore
reader_output_df.insert(5, "Sample", reader_output_df["SiteSample"].str.extract("([\d.]+)", expand=True))  # type: ignore

reader_output_df.drop(columns=["SiteSample"], inplace=True)
reader_output_df.drop(columns=["Sample ID"], inplace=True)

reader_output_df.drop(columns=["Barcode Text", "Plate Type"], inplace=True)

# drop useless columns
reader_output_df.drop(
    columns=[
        "Red With Gas Raw Count",
        "Red With Gas Edited Count",
        "Red Without Gas Raw Count",
        "Red Without Gas Edited Count",
        "Red Without Gas Calculated Result",
        "Blue With Gas Raw Count",
        "Blue With Gas Edited Count",
        "Blue Without Gas Raw Count",
        "Blue Without Gas Edited Count",
        "Blue Without Gas Calculated Result",
        "Comments",
    ],
    inplace=True,
)

reader_output_df.rename(
    {
        "Red With Gas Calculated Result": "Coliform (1ml)",
        "Blue With Gas Calculated Result": "Ecoli (1ml)",
    },
    axis=1,
    inplace=True,
)

reader_output_df["Sample"] = reader_output_df["Sample"].astype(float)
reader_output_df["Site"] = reader_output_df["Site"].str.upper()

reader_output_df.dropna(subset=["Coliform (1ml)", "Ecoli (1ml)"], inplace=True)


reader_output_df

/var/folders/5l/c1vg2tp11w33f10cy5zpdxcc0000gn/T/ipykernel_58089/2257464726.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  reader_output_df["Image Date Time"] = pd.to_datetime(


,Technician,Date,Time,Site,Sample,Image Date Time,Dilution,Coliform (1ml),Ecoli (1ml)
0,Admin,2023-09-05,10:10,B,1.1,2023-10-13 13:05:35,1:1,334,7
1,Admin,2023-09-05,10:10,B,1.2,2023-10-13 13:06:12,1:1,364,9
2,Admin,2023-09-05,10:10,B,2.1,2023-10-13 13:06:57,1:1,26,4
3,Admin,2023-09-05,10:10,B,2.2,2023-10-13 13:07:50,1:1,43,10
4,Admin,2023-09-05,11:26,B,1.1,2023-10-13 13:09:26,1:1,36,5
...,...,...,...,...,...,...,...,...,...
2169,Admin,2023-08-24,14:05,D,2.2,2023-08-30 14:41:47,1:1,52,15
2170,Admin,2023-08-24,14:20,E,1.2,2023-08-30 14:43:18,1:1,25,6
2171,Admin,2023-08-24,14:20,E,1.1,2023-08-30 14:45:21,1:1,34,9
2172,Admin,2023-08-24,14:20,E,2.1,2023-08-30 14:46:30,1:1,49,4


In [13]:
raw_extra_data_path = "/Users/massimilianoarca/Library/CloudStorage/OneDrive-PolitecnicodiMilano/Thames Ecoli/2023_08_10_AM_pt2.csv"
raw_extra_data_df = pd.read_csv(raw_extra_data_path, sep=";")

In [14]:
extra_data_df = raw_extra_data_df.copy()

extra_data_df.drop_duplicates(inplace=True)

extra_data_df[["Date", "Time", "SiteSample"]] = raw_extra_data_df[
    "Sample ID"
].str.split("_", expand=True)

extra_data_df["Date"] = pd.to_datetime(
    extra_data_df["Date"], format="%Y%m%d"
).dt.date
extra_data_df["Time"].replace("XXXX", pd.NaT, inplace=True)  # type: ignore
extra_data_df["Time"] = pd.to_datetime(
    extra_data_df["Time"], format="%H%M", errors="coerce"
).dt.strftime("%H:%M")
extra_data_df["Image Date Time"] = pd.to_datetime(
    extra_data_df["Image Date Time"]
)

date_col = extra_data_df.pop("Date")
time_col = extra_data_df.pop("Time")

extra_data_df.insert(2, "Date", date_col)
extra_data_df.insert(3, "Time", time_col)

extra_data_df.insert(4, "Site", extra_data_df["SiteSample"].str.extract("([A-Za-z]+)", expand=True))  # type: ignore
extra_data_df.insert(5, "Sample", extra_data_df["SiteSample"].str.extract("([\d.]+)", expand=True))  # type: ignore

extra_data_df.drop(columns=["SiteSample"], inplace=True)
extra_data_df.drop(columns=["Sample ID"], inplace=True)
extra_data_df.drop(columns=["Plate Type"], inplace=True)

extra_data_df["Sample"] = extra_data_df["Sample"].astype(float)
extra_data_df["Site"] = extra_data_df["Site"].str.upper()

extra_data_df.dropna(subset=["Coliform (1ml)", "Ecoli (1ml)"], inplace=True)

In [15]:
reader_output_df = pd.concat([reader_output_df, extra_data_df], ignore_index=True)
reader_output_df.drop_duplicates(inplace=True)

### Raw Manual Counting

In [16]:
manual_counting_df = raw_manual_counting_df.copy()

manual_counting_df.drop_duplicates(inplace=True)

manual_counting_df["Sample"] = manual_counting_df["Sample"].astype(float)
manual_counting_df.rename(
    {"Counter": "Technician"},
    axis=1,
    inplace=True,
)

manual_counting_df["Date"] = manual_counting_df["Date"].dt.date
manual_counting_df["Time"] = pd.to_datetime(
    manual_counting_df["Time"], format="%H:%M:%S", errors="coerce"
).dt.strftime("%H:%M")

manual_counting_df.dropna(
    subset=["Coliform (1ml)", "Ecoli (1ml)"], inplace=True
)

In [17]:
manual_counting_df

,Technician,Date,Time,Site,Sample,Ecoli (1ml),Coliform (1ml)
0,Randa,2023-01-31,10:40,A,1.0,3.0,13.0
1,Randa,2023-01-31,10:40,A,2.0,3.0,7.0
2,Randa,2023-01-31,11:14,C,2.0,6.0,11.0
3,Randa,2023-01-31,11:15,C,1.0,8.0,11.0
4,Randa,2023-01-31,11:35,D,1.0,11.0,10.0
...,...,...,...,...,...,...,...
165,Alisha,2023-06-29,11:47,B,1.1,241.0,131.0
166,Alisha,2023-06-29,11:22,A,1.2,81.0,16.0
167,Alisha,2023-06-29,11:22,A,1.1,95.0,12.0
168,Alisha,2023-06-29,11:22,A,2.2,42.0,6.0


### Raw Water Quality

In [18]:
water_quality_df = raw_water_quality_df.copy()

water_quality_df["Sample"] = water_quality_df["Sample"].astype(float)
water_quality_df.rename(
    {"Sampler": "Technician"},
    axis=1,
    inplace=True,
)

water_quality_df = water_quality_df[water_quality_df["FOLLOW UP"] != "X"]
water_quality_df["Date"] = water_quality_df["Date"].dt.date
water_quality_df["Time"] = pd.to_datetime(
    water_quality_df["Time"], format="%H:%M:%S", errors="coerce"
).dt.strftime("%H:%M")

water_quality_df.drop(columns=["FOLLOW UP"], inplace=True)
water_quality_df.dropna(
    subset=["Temp C", "Ph", "Cond (ms)"], inplace=True
)

In [19]:
water_quality_df

,Technician,Date,Time,Site,Sample,Temp C,Ph,Cond (ms)
0,Randa,2023-01-31,10:40,A,1.0,10.0,8.40,0.69
1,Randa,2023-01-31,10:40,A,2.0,9.5,8.20,0.73
2,Randa,2023-01-31,11:15,C,1.0,8.8,8.20,0.72
3,Randa,2023-01-31,11:15,C,2.0,7.9,8.10,0.71
8,Randa,2023-01-31,11:55,E,1.0,8.1,7.97,0.79
...,...,...,...,...,...,...,...,...
687,NaN,2023-09-07,15:27,D,2.0,25.0,7.90,1.46
688,NaN,2023-09-07,15:32,A,1.0,27.7,8.00,1.86
689,NaN,2023-09-07,15:32,A,2.0,26.5,8.00,1.89
690,NaN,2023-09-07,15:45,E,1.0,25.0,7.80,1.52


## Merge Data

In [20]:
reader_manual_df = pd.concat(
    [reader_output_df, manual_counting_df], ignore_index=True
)

In [21]:
full_df = pd.merge(
    reader_manual_df,
    water_quality_df,
    how="outer",
    on=["Technician", "Date", "Time", "Site", "Sample"],
)

## Final Cleaning

In [22]:
full_df["Coliform (1ml)"].replace(to_replace="TNTC", value=250, inplace=True)
full_df["Ecoli (1ml)"].replace(to_replace="TNTC", value=250, inplace=True)

full_df["Technician"].replace(to_replace=", ", value=" - ", inplace=True, regex=True)

In [23]:
full_df

,Technician,Date,Time,Site,Sample,Image Date Time,Dilution,Coliform (1ml),Ecoli (1ml),Temp C,Ph,Cond (ms)
0,Admin,2023-09-05,10:10,B,1.1,2023-10-13 13:05:35,1:1,334,7,NaN,NaN,NaN
1,Admin,2023-09-05,10:10,B,1.2,2023-10-13 13:06:12,1:1,364,9,NaN,NaN,NaN
2,Admin,2023-09-05,10:10,B,2.1,2023-10-13 13:06:57,1:1,26,4,NaN,NaN,NaN
3,Admin,2023-09-05,10:10,B,2.2,2023-10-13 13:07:50,1:1,43,10,NaN,NaN,NaN
4,Admin,2023-09-05,11:26,B,1.1,2023-10-13 13:09:26,1:1,36,5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2966,NaN,2023-09-07,15:27,D,2.0,NaT,NaN,NaN,NaN,25.0,7.9,1.46
2967,NaN,2023-09-07,15:32,A,1.0,NaT,NaN,NaN,NaN,27.7,8.0,1.86
2968,NaN,2023-09-07,15:32,A,2.0,NaT,NaN,NaN,NaN,26.5,8.0,1.89
2969,NaN,2023-09-07,15:45,E,1.0,NaT,NaN,NaN,NaN,25.0,7.8,1.52


In [24]:
full_df.to_csv(os.path.join(dir_store_path, "full_dataset.csv"), index=False)

## Site Positions

In [25]:
all_overflow_sites_path = "/Users/massimilianoarca/Library/CloudStorage/OneDrive-PolitecnicodiMilano/Thames Ecoli/sites/all_overflows.csv"
nearest_overflow_sites_path = "/Users/massimilianoarca/Library/CloudStorage/OneDrive-PolitecnicodiMilano/Thames Ecoli/sites/nearest_overflows.csv"
sampling_sites_path = "/Users/massimilianoarca/Library/CloudStorage/OneDrive-PolitecnicodiMilano/Thames Ecoli/sites/sampling.csv"

In [26]:
all_overflow_sites_df = pd.read_csv(all_overflow_sites_path)
nearest_overflow_sites_df = pd.read_csv(nearest_overflow_sites_path)
sampling_sites_df = pd.read_csv(sampling_sites_path)

In [45]:
final_df = full_df.copy()

In [46]:
final_df["Coliform (1ml)"] = final_df["Coliform (1ml)"].astype(float)
final_df["Ecoli (1ml)"] = final_df["Ecoli (1ml)"].astype(float)

In [47]:
final_df = final_df.groupby(["Technician", "Date", "Time", "Site", "Sample"], as_index=False).agg(
    {
        'Coliform (1ml)': ["mean", "std"],
        'Ecoli (1ml)': ["mean", "std"],
        'Temp C': ["mean", "std"],
        'Ph': ["mean", "std"],
        'Cond (ms)': ["mean", "std"],
    }
    ,
)

In [50]:
final_df

Technician        Date   Time Site Sample Coliform (1ml)            \
                                                         mean       std   
0         Admin  2023-08-14  10:10    A    1.1      95.000000  1.732051   
1         Admin  2023-08-14  10:10    A    1.2     102.000000  1.000000   
2         Admin  2023-08-14  10:10    A    2.1      70.666667  0.577350   
3         Admin  2023-08-14  10:10    A    2.2      77.666667  1.154701   
4         Admin  2023-08-14  10:10    B    1.1      53.666667  1.527525   
...         ...         ...    ...  ...    ...            ...       ...   
1486        Vee  2023-09-06  15:08    C    2.2      23.000000       NaN   
1487        Vee  2023-09-06  15:25    D    1.1      21.000000       NaN   
1488        Vee  2023-09-06  15:25    D    1.2      23.000000       NaN   
1489        Vee  2023-09-06  15:25    D    2.1      28.000000       NaN   
1490        Vee  2023-09-06  15:25    D    2.2      26.000000       NaN   

     Ecoli (1ml)      Temp C       Ph     Cond (ms)      
            mean  std   mean std mean std      mean std  
0            2.0  0.0    NaN NaN  NaN NaN       NaN NaN  
1            8.0  0.0    NaN NaN  NaN NaN       NaN NaN  
2            8.0  1.0    NaN NaN  NaN NaN       NaN NaN  
3            9.0  0.0    NaN NaN  NaN NaN       NaN NaN  
4            4.0  1.0    NaN NaN  NaN NaN       NaN NaN  
...          ...  ...    ...  ..  ...  ..       ...  ..  
1486         3.0  NaN    NaN NaN  NaN NaN       NaN NaN  
1487         3.0  NaN    NaN NaN  NaN NaN       NaN NaN  
1488         1.0  NaN    NaN NaN  NaN NaN       NaN NaN  
1489         4.0  NaN    NaN NaN  NaN NaN       NaN NaN  
1490         2.0  NaN    NaN NaN  NaN NaN       NaN NaN  

[1491 rows x 15 columns]